In [1]:
from codelin.models.const_tree import C_Tree
from codelin.encs.constituent import *
from codelin.utils.constants import *
from codelin.models.linearized_tree import LinearizedTree
from codelin.models.const_label import C_Label
from nltk.tree import Tree

import pandas as pd
import os
import copy

def pt(t, d=True):
    '''
    Print tree if d is True
    '''
    if d:
        Tree.fromstring(str(t)).pretty_print()

def bin_print(ts):
    '''
    Print binary tree
    '''
    Tree.fromstring(str(C_Tree.to_binary_right(C_Tree.from_string(ts)))).pretty_print()

tsa = "(NTA (PA a) (NTB (NTC (NTD (PB b) (NTE (PC c) (NTF (NTG (PD d) (PE e)) (PF f)))) (PG g)) (PH h)))"
tsb = "(NTA (PA a) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"
tsc = "(NTA (NTU (NTV (PA11 a11) (PA12 a12)) (NTW (PA21 a21) (PA22 a22))) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"
tsd = "(NTA (NTU (NTV (NTP (S s) (T t)) (PA12 a12)) (NTW (PA21 a21) (NTQ (X x) (Y y)))) (NTB (NTC (PB b) (NTF (NTD (PC c) (NTE (PD d) (PE e))) (PF f))) (NTG (NTH (PG g) (NTI (PH h) (NTJ (PI i) (PJ j)))) (PK k))))"

ts1 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts2 = "(SINV (S (ADVP (RB Once) (RB again)) (-LRB- -LCB-) (NP (DT the) (NNS specialists)) (-RRB- -RCB-) (VP (VBD were) (RB not) (ADJP (JJ able) (S (VP (TO to) (VP (VB handle) (NP (NP (DT the) (NNS imbalances)) (PP (IN on) (NP (NP (DT the) (NN floor)) (PP (IN of) (NP (DT the) (NNP New) (NNP York) (NNP Stock) (NNP Exchange)))))))))))) (, ,) ('' '') (VP (VBD said)) (NP (NP (NNP Christopher) (NNP Pedersen)) (, ,) (NP (NP (JJ senior) (NN vice) (NN president)) (PP (IN at) (NP (NNP Twenty-First) (NNP Securities) (NNP Corp))))) (. .))"
ts3 = "(S (NP (NP (NNP Seven) (NNP Big) (NNP Board) (NNS stocks)) (: --) (NP (NP (NNP UAL)) (, ,) (NP (NNP AMR)) (, ,) (NP (NNP BankAmerica)) (, ,) (NP (NNP Walt) (NNP Disney)) (, ,) (NP (NNP Capital) (NNP Cities\/ABC)) (, ,) (NP (NNP Philip) (NNP Morris)) (CC and) (NP (NNP Pacific) (NNP Telesis) (NNP Group))) (: --)) (VP (VP (VBD stopped) (S (VP (NN trading)))) (CC and) (VP (ADVP (RB never)) (VBD resumed))) (. .))"
ts4 = "(S (S (NP (JJ Big) (NN investment) (NNS banks)) (VP (VBD refused) (S (VP (TO to) (VP (VB step) (ADVP (RB up) (PP (TO to) (NP (DT the) (NN plate)))) (S (VP (TO to) (VP (VB support) (NP (DT the) (JJ beleaguered) (NN floor) (NNS traders)) (PP (IN by) (S (VP (VBG buying) (NP (NP (JJ big) (NNS blocks)) (PP (IN of) (NP (NN stock))))))))))))))) (, ,) (NP (NNS traders)) (VP (VBP say)) (. .))"
ts5 = "(FRAG (SBAR (IN As) (IN in) (: :) (`` ``) (SQ (NP (PRP You)) (VP (VBD went) (NP (VBG ballooning))) (. ?) (. ?) (. !) (. !))))"
ts6 = "(NP (NP (NN Year)) (VP (VBN ended) (NP (NNP Dec.) (CD 31) (, ,) (CD 1988))) (X (NN \*)))"
ts7 = "(S (INTJ (RB No)) (, ,) (NP (PRP it)) (VP (VBD was) (RB n't) (NP (NNP Black) (NNP Monday))) (. .))"
ts8 = "(S (NP (DT The) (JJ finger-pointing)) (VP (VBZ has) (ADVP (RB already)) (VP (VBN begun))) (. .))"
ts9 = "(S (`` ``) (NP (DT The) (NN equity) (NN market)) (VP (VBD was) (ADJP (JJ illiquid))) (. .))"
ts10 = "(S (PP (IN In) (NP (NP (DT an) (NNP Oct.) (CD 19) (NN review)) (PP (IN of) (NP (`` ``) (NP (DT The) (NN Misanthrope)) ('' '') (PP (IN at) (NP (NP (NNP Chicago) (POS 's)) (NNP Goodman) (NNP Theatre))))) (PRN (-LRB- -LRB-) (`` ``) (S (NP (JJ Revitalized) (NNS Classics)) (VP (VBP Take) (NP (DT the) (NNP Stage)) (PP (IN in) (NP (NNP Windy) (NNP City))))) (, ,) ('' '') (NP (NNP Leisure) (CC &) (NNP Arts)) (-RRB- -RRB-)))) (, ,) (NP (NP (NP (DT the) (NN role)) (PP (IN of) (NP (NNP Celimene)))) (, ,) (VP (VBN played) (PP (IN by) (NP (NNP Kim) (NNP Cattrall)))) (, ,)) (VP (VBD was) (VP (ADVP (RB mistakenly)) (VBN attributed) (PP (TO to) (NP (NNP Christina) (NNP Haag))))) (. .))"

/home/poli/.conda/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Gaps Playground

In [11]:
def empty_preterminal():
    return C_Tree(label = "EMPTY", children=C_Tree(label = "EMPTY"))
def empty_tree():
    return C_Tree(label = "EMPTY", children=[empty_preterminal(), empty_preterminal()])
def insert_children_on_empty_child(tree, child):
    if tree.children[0].label == "EMPTY":
        tree.children[0] = child
    else:
        tree.children[1] = child
    tree.update_custody()
def is_subtree(t1, t2):
    t1t = t1.get_terminals()
    t2t = t2.get_terminals()
    return all (item in t1t for item in t2t) or all (item in t2t for item in t1t)
def is_closed(tree):
    print("checking isclosed for", tree)
    # Base case: an empty tree has all empty children
    if not tree.children:
        return False

    # Recursive case: check all child nodes
    for child in tree.children:
        if child.label != 'EMPTY':
            return True
        if not is_closed(child):
            return True

    return False
def has_empty(tree):
    return "EMPTY" in str(tree)



# Operators

def operator_g(can, stack, word, non_terminal):        
    # Crear un camino hacia abajo El no terminal se pone al final de ese camino
    can = C_Tree(label=non_terminal, children=[empty_preterminal(),empty_preterminal()])

    # Enlazamos la palabra actual como su hijo izquierdo
    can.children[0] = C_Tree(label="P", children=C_Tree(label=word))
    can.update_custody()
    
    # Lo activamos
    while can.parent is not None and can.parent.children[1].label != "EMPTY":
        can = can.parent
    print("==> Adding to current active node stacks the following tree:")
    Tree.fromstring(str(can)).pretty_print()
    stack.append(can)
    


def operator_n1(can, stack, word, non_terminal):
    can = stack.pop()
    while len(can.children)>0 and can.children[1].label != "EMPTY":
        can = can.children[1]
    # Enlazar la palabra actual como hijo derecho del nodo activo.
    can.children[1] = C_Tree(label="P", children=C_Tree(label=word))
    can.update_custody()
    # Después, un nivel por encima del nodo activo, añadir el no terminal dado (vamos, lo añadimos como padre del activo) 
    can = C_Tree(label=non_terminal, children=[can, empty_preterminal()])
    stack.append(can)
    


def operator_1(can, stack, word, non_terminal):
    can = stack.pop()

    while len(can.children)>0 and can.children[1].label != "EMPTY":
        can = can.children[1]
    
    # Al nodo activo, que ya debería tener hijo izquierdo, le añadimos un hijo derecho con el no terminal dado
    can.children[1] = C_Tree(label=non_terminal, children=[empty_preterminal(), empty_preterminal()])
    can.update_custody()
    # La palabra actual se enlaza como hijo izquierdo de este último.

    can.children[1].children[0] = C_Tree(label="P", children=C_Tree(label=word))
    can.update_custody()

    while can.parent is not None:
        can = can.parent

    # Y lo activamos.
    stack.append(can)
    
def operator_ng(can, stack, word, non_terminal):
    can = stack.pop()
    print("Current Active Subtree (word will be inserted in the EMPTY node)")
    Tree.fromstring(str(can)).pretty_print()
    
    # Insert the word in the first empty node
    while len(can.children)>0 and can.children[1].label != "EMPTY":
        can = can.children[1]
    can.children[1] = C_Tree(label="P", children=C_Tree(label=word))
    can.update_custody()
    
    if non_terminal == "$$":
        while len(stack)>0:
            stack_top = stack.pop()
            print("$S stack_top")
            Tree.fromstring(str(stack_top)).pretty_print()
            print("$$ can")
            Tree.fromstring(str(can)).pretty_print()
            while len(stack_top.children)>0 and stack_top.children[1].label != "EMPTY":
                stack_top = stack_top.children[1]
            stack_top.children[1] = can
            print("$$ results")
            Tree.fromstring(str(stack_top)).pretty_print()
            can = stack_top
        stack.append(stack_top)
        return 
        
    # get the top of the current active node
    print("What are we inserting? (Top of the Current Active Subtree) =>")
    while can.parent is not None:
        can = can.parent
    Tree.fromstring(str(can)).pretty_print()
    node_to_insert = can
    
    # if we have more open gaps
    if len(stack)>1:
        last_open_gap = stack.pop()
        print("Last open gap in the stack =>", last_open_gap)
        Tree.fromstring(str(last_open_gap)).pretty_print()

        # get the bottom of the last open gap
        print("Where are we inserting? (Bottom of last open gap) =>")
        target_insert = last_open_gap
        while len(target_insert.children)>0 and target_insert.children[1].label!="EMPTY":
            target_insert = target_insert.children[1]
        Tree.fromstring(str(target_insert)).pretty_print()

        print("Inserting the Current Active Subtree into the target_insert of the last_open_gap =>")
        target_insert.children[1] = can
        target_insert.update_custody()
        Tree.fromstring(str(last_open_gap)).pretty_print()
        
        node_to_insert = last_open_gap

    print("Inserting the non-terminal as parent =>")
    can = C_Tree(label=non_terminal, children=[node_to_insert, empty_preterminal()])
    Tree.fromstring(str(can)).pretty_print()


    print("Merging Current Active Subtree with top of the stack =>")
    if len(stack)>1:
        stack_top = stack.pop()
        while len(stack_top.children)>0 and stack_top.children[1].label != "EMPTY":
            stack_top = stack_top.children[1]
        stack_top.children[1] = can
        stack_top.update_custody()
        Tree.fromstring(str(stack_top)).pretty_print()
        can = stack_top
    Tree.fromstring(str(can)).pretty_print()

    stack.append(can)


debug = False
encoder = C_NaiveRelativeEncoding(separator="_",
                                    unary_joiner="+",
                                    reverse=False,
                                    binary=True,
                                    binary_direction="R",
                                    binary_marker="*")
tencoder = C_Tetratag(separator="_",
                        unary_joiner="+",
                        reverse=False,
                        binary_direction="R",
                        binary_marker="*")

# ptb_path = "/home/poli/Documents/Treebanks/PENN_TREEBANK/train.trees"
ptb_path="/home/poli/Treebanks/const/PENN_TREEBANK/dev.trees"
ptb_file = open(ptb_path, "r")


# good examples: 2,6,9
# bad example: 4

idx     = 4
use_n1  = True
for line in ptb_file.readlines()[idx:idx+1]:
    tree = C_Tree.from_string(line.rstrip())

    lc_tree = encoder.encode(copy.deepcopy(tree))
    lc_tree_tetratag = tencoder.encode(copy.deepcopy(tree))

    # get the new labels
    new_labels = []
    for label, tetra_label in zip(lc_tree.labels, lc_tree_tetratag.labels):        
        new_nc = ""
        # First label will always be a 'open gap' operator
        if lc_tree.labels.index(label)==0:
            new_nc = "G"
        
        # Last label will always be a 'close gap' operator
        elif lc_tree.labels.index(label)==len(lc_tree.labels)-1:
            new_nc = "-G"
        
        # Otherwise, assign the number of common nodes
        elif label.n_commons == 1:
            new_nc = "1"
        elif label.n_commons == -1:
            new_nc = "-1"
        elif label.n_commons < 0:
            new_nc = "-G"
        else:
            new_nc = "G"
        
        lc = label.last_common if lc_tree.labels.index(label)!=len(lc_tree.labels)-1 else "$$"
        new_labels.append((new_nc, lc, label.n_commons, tetra_label.n_commons))


      
    print("Target Tree:")

    # collapse unary tags
    tree = tree.collapse_unary()

    # replace the preterminals with P
    for node in tree.get_preterminals():
        node.label = "P"

    tree = C_Tree.to_binary_right(tree)
    pt(tree)
    print("\n\n")
    print("All labels:\n")
    print(f"{'word':<15}  {'non_terminal':<15} {'label (rel)':<15} {'label (4ec)':<15} {'label (gaps)':<15}")
    for label, word in zip(new_labels, lc_tree.words):
        print(f"{word:<15}  {label[1]:<15} {label[0]:<15} {label[2]:<15} {label[3]:<15}")
    print("\n\n")
    # build the tree
    stack     = []  
    active_node = None
    for label, word in zip(new_labels, lc_tree.words):
        print("=============================================")
        print(f"{'word':<15}  {'non_terminal':<15} {'label (rel)':<15} {'label (4ec)':<15} {'label (gaps)':<15}")
        print(f"{word:<15}  {label[1]:<15} {label[0]:<15} {label[2]:<15} {label[3]:<15}")
        print("STACK: [")
        for s in stack:
            print('\t\t'+str(s))
        print("]")
        print("=============================================")
        
        if label[0] == "1":
            operator_1(active_node, stack, word, label[1])
        elif label[0] == "-1":
            operator_n1(active_node, stack, word, label[1])
        elif label[0] == "G":
            operator_g(active_node, stack, word, label[1])
        elif label[0] == "-G":
            operator_ng(active_node, stack, word, label[1])
        else:
            raise ValueError("Unknown label: {}".format(label))

        print("Topmost parent of the Current Active Node after processing label =>")
        active_node = stack[-1]
        while active_node.parent is not None:
            active_node = active_node.parent
        Tree.fromstring(str(active_node)).pretty_print()

    print("## Done with remaining Stack Size =", len(stack))
    print("Original:")
    pt(tree)
    print("Result (Last Active Subtree):")
    pt(active_node)
    print(str(tree))
    print(str(active_node))
    assert str(tree)==str(active_node), "not equal"

Target Tree:
                                                                                     S                                                                           
                                     ________________________________________________|______________________________                                              
                                    |                                                                               S*                                           
                                    |                                           ____________________________________|__________________________________________   
                                    |                                          VP                                                                              | 
                                    |                                  ________|_____                                                                          |  
            

AssertionError: not equal

In [ ]:
raise ValueError("Done")

ValueError: Done

## Labels equivalence

In [ ]:
ptb_path = "~/Treebanks/const/PENN_TREEBANK/"
ptb_path = os.path.expanduser(ptb_path)
print("Reading PTB from",ptb_path)    
df = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
with open(os.path.join(ptb_path,"test.trees")) as f:
    ptb = f.read().splitlines()

for tree_string in ptb:
    tree = C_Tree.from_string(tree_string)
    rel_encoder = C_NaiveRelativeEncoding(separator="_", unary_joiner="+", reverse=False, binary=True, binary_marker="[*]", binary_direction="R")
    tet_encoder = C_Tetratag(separator="_", unary_joiner="+", reverse=False, binary_marker="[*]", binary_direction="R")

    t_lin_tree = tet_encoder.encode(copy.deepcopy(tree))
    r_lin_tree = rel_encoder.encode(copy.deepcopy(tree))

    df_temp = pd.DataFrame(columns=["Word", "Relative", "Tetratag"])
    
    df_temp["Word"] = r_lin_tree.words
    df_temp["Relative"] = [str(l.n_commons) for l in r_lin_tree.labels]
    df_temp["Tetratag"] = [l.n_commons for l in t_lin_tree.labels]
    
    df = pd.concat([df, df_temp], axis=0)

    r_decoded_tree = rel_encoder.decode(r_lin_tree)
    r_decoded_tree = r_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
    
    t_decoded_tree = tet_encoder.decode(t_lin_tree)
    t_decoded_tree = t_decoded_tree.postprocess_tree(conflict_strat=C_STRAT_MAX, clean_nulls=True)
        
    # if not (str(r_decoded_tree)==tree_string):
    #     print("Decoded tree is different from original")
    #     print("Original:",tree_string)
    #     print("Decoded:",r_decoded_tree)

Reading PTB from /home/poli/Treebanks/const/PENN_TREEBANK/


In [ ]:
# print relative values for each of the unique values of tetratag
tetra_values = df["Tetratag"].unique()
tetra_values.sort()
for tetratag in tetra_values:
    # ignore 'close' labels
    if len(tetratag)==1:
        continue
    
    # sort
    rel_values = df[df["Tetratag"]==tetratag]["Relative"].unique()
    rel_values = [int(v) for v in rel_values]
    rel_values.sort()
    print("Tetratag:",tetratag,"\n\t==> Relative:",rel_values)

Tetratag: lL 
	==> Relative: [-31, -30, -27, -26, -25, -24, -23, -22, -21, -20, -19, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
Tetratag: lR 
	==> Relative: [-21, -20, -18, -17, -16, -15, -14, -13, -12, -11, -10, -9, -8, -7, -6, -5, -4, -3, -2, -1]
Tetratag: rL 
	==> Relative: [1]
Tetratag: rR 
	==> Relative: [1, 2, 3, 4, 5]
